# Introduction

## What is Pennylane?
Pennylane is a library for programming quantum computers which allows the execution of quantum programs on various backends.
<br>
Pennylane also integrates with other popular Python libraries such as NumPy and PyTorch.


### Documentation:
https://docs.pennylane.ai/en/stable/introduction/circuits.html#device-options
### API Reference:
https://docs.pennylane.ai/en/stable/code/qml.html

# Setup
To setup your environment, please run the provided setup.py script yourself OR run the next cell

In [ ]:
!python setup.py

In [ ]:
!python verify_install.py

If your environment is ready, you will have seen a SUCCESS message. <br>
Also run this next cell to test the notebook.

In [ ]:
import pennylane as qml
# We'll also need PENNYLANE's numpy wrapper
from pennylane import numpy as np
# And matplotlib is used for some helper visuals
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
print(qml.__version__)

# Syntax

## Pennylane is backend agnostic. How does it abstract it away?

In [ ]:
# Wires == No. Qubits
dev = qml.device('default.qubit', wires=2)

Parameters:
  - Name: needs a string to identify the type of devices you run on
    - Is it a simulator or a real quantum computer
  - To see the possibilities, consult the docs: https://docs.pennylane.ai/en/stable/code/qml_devices.html <br> or use qml.about() to see the devices avaialble on your system

In [ ]:
qml.about()

### Wires


Wires are simple labels for qubits

The easiest way to define a lot of wires is by specifying how many you need
<br> The wires will be <span style="color:orange"> 0 indexed </span>

In [ ]:
# This makes a circuit with wires 0, 1 and 2
dev_wires = qml.device('default.qubit', wires=3)

Some times,
<br>  _especially when making a presentation ;)_ <br>
you might want to label the wires with some names.
<br> With pennylane, you can.
<br> We all love our friends <span style="color:green">**Alice** and **Bob**</span>, so I'll ask them for help here

In [ ]:
dev_wires = qml.device('default.qubit', wires=['Alice', 'Bob'])
@qml.qnode(dev_wires)
def entangle_alice_and_bob():
    qml.Hadamard(wires='Alice')
    qml.CNOT(wires=['Alice', 'Bob'])
    return qml.state()

entangle_alice_and_bob()

### Shots

Pennylane offers two main ways of running circuits:
  - in their general mathematical sense
  - using "*shots*", where you will have the randomness that comes with quantum hardware(either simulated or realistic)

The default for simulated devices is None. <br>
For the hardware plugins, it varies by provider.

In [ ]:
dev_ex_shots = qml.device("default.qubit", wires=1)

# you can always check how many shots a devices has by doing
print(f"Device has: {dev_ex_shots.shots} shots")


The behavior above is getting deprecated, we'll see the modern approach below

#### Good to know:
if you run your code with shots on a real quantum computer(eg IBM's, AWS Bracket, etc)
<br> YOU PAY FOR EVERY SHOT

In [ ]:
#We'll define a simple circuit and see what difference this makes
def shots_circuit():
    qml.Hadamard(wires=0)
    return qml.expval(qml.PauliZ(0))

#### Mathematical

In [ ]:
dev_no_shots = qml.device("default.qubit", wires=1, shots=None)

In [ ]:
no_shots_circuit = qml.QNode(shots_circuit, dev_no_shots)
no_shots_circuit()

#### Simulated / Hardware

In [ ]:
dev_with_shots = qml.device("default.qubit", wires=1)

In [ ]:
shots_circuit_ex = qml.QNode(shots_circuit, dev_with_shots, shots=1000)
# the output here will be the average of the 1000 shots
shots_circuit_ex()

In [ ]:
#To address the mention above, regarding the api change
shots_circuit_ex.shots
# is how you find out the number of shots if it is defined in the Qnode, not the device ( the new standard)

### How to get results?

In [ ]:
#Might give a dep warning, but the api is fine for this case
dev_result = qml.device("default.qubit", wires=1, shots=10)

In [ ]:
@qml.qnode(dev_result)
def measurement_demo():
    qml.Hadamard(wires=0)
    # Try swapping these out to see the difference!
    # return qml.probs(wires=0)   # Theoretical probabilities
    # return qml.expval(qml.PauliZ(0)) # Expectation value
    return qml.sample(qml.PauliZ(0))   # Raw measurement shots

print(f"Measurement results: {measurement_demo()}")

## Circuit Templates

**Templates** are pre-built circuit patterns that encapsulate common quantum operations. They provide:

- **Standardized building blocks** for quantum circuits
- **Cleaner, more readable code** for complex patterns
- **Community-recognized structures** (e.g., for QML papers/tutorials)

In [ ]:
dev_templates = qml.device('default.qubit', wires=5)


features = np.random.random(5)

In [ ]:
#The dyi python way
@qml.qnode(dev_templates)
def loop_encoding(features):
    for i in range(5):
        qml.RX(features[i], wires=i)
    return qml.state()


In [ ]:
# Template way
@qml.qnode(dev_templates)
def template_encoding(features):
    qml.AngleEmbedding(features, wires=range(5))
    return qml.state()

### Common Template Categories

- **Embeddings**: Encode classical data (`AngleEmbedding`, `AmplitudeEmbedding`)
- **Layers**: Variational ansätze (`StronglyEntanglingLayers`, `BasicEntanglerLayers`)
- **State preparations**: Initialize specific quantum states

Templates are a bit too complex for the purpose of this lab, but you are welcome to explore them further. <br>
**Explore more**: [PennyLane Templates Documentation](https://docs.pennylane.ai/en/stable/introduction/templates.html)

## Running the same circuit with multiple devices

Because of practical constraints, we'll focus on the local models during the lab.
<br>

In [ ]:
# Lets first define the a list of devices.
# As they are strings, it is trivial
devices = ['default.qubit','lightning.qubit', 'default.clifford']

In [ ]:
#################################################################
#Ignore this block,
#just prints a visual representation of the circuit to help out
#################################################################
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#image from https://upload.wikimedia.org/wikipedia/en/thumb/5/57/The_quantum_logic_gates_that_generates_the_3-qubit_GHZ_state.png/330px-The_quantum_logic_gates_that_generates_the_3-qubit_GHZ_state.png
ghz_circuit = mpimg.imread("images/GHZ_state.png")
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(ghz_circuit)


Reminder: to use any gate you can do qml.GATE_NAME
<br>
For example: qml.Hadamard(wires=0)

In [ ]:
# As our base circuit, we'll use one to generate a 3 qubit entagled state(GHZ state)
def ghz_circuit():
    # TODO: Implement the circuit above
    return qml.counts()

In [ ]:
for dev_name in devices:
    dev = qml.device(dev_name, wires=3, shots=1000)
    qnode = qml.QNode(ghz_circuit, dev)
    counts = qnode()
    print(f"Device: {dev_name} | Counts: {counts}")

# Simple Exercises

Let's start with a simple exercise to see how we work with pennylane

## 1. Use rotations to take a |0> and rotate it such that its projection on the Z axis is 0.

### Visual explanation:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img1 = mpimg.imread('images/bloch_init.jpg')
img2 = mpimg.imread('images/Bloch-x rotated.jpg')
img3 = mpimg.imread('images/Bloch-xy-rotated.jpg')
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))


axes[0].imshow(img1)
axes[0].set_title("Original Image")
axes[0].axis('off')
axes[1].imshow(img2)
axes[1].set_title("After X Rotation")
axes[1].axis('off')
axes[2].imshow(img3)
axes[2].set_title("After X and Y Rotation")
axes[2].axis('off')


### Implementation

In [ ]:
dev = qml.device("default.qubit", wires=1, shots=1000)

In [ ]:
@qml.qnode(dev)
def rotate_qbit(phi, theta):
    #TODO:
    return qml.expval(qml.PauliZ(0))

In [ ]:
phi, theta = None, None
result = rotate_qbit(phi, theta)
print(result)


In [ ]:
assert(np.isclose(result, 0, atol=1e-10))

### BONUS: find an alternative way to call your function such that it gives the same result

In [ ]:
phi, theta = None, None
result2 = rotate_qbit(phi, theta)
print(result2)

 ## 2. Basics of QML(Quantum Machine Learning)

### Making a model to force the state of a qubit to state |1>

In [ ]:
dev = qml.device("default.qubit", wires=1)

In [ ]:
# 1. THE MODEL (The Neural Network equivalent)
# Instead of neurons, we have wires. Instead of weights, we have rotation angles.
@qml.qnode(dev)
def quantum_model(weights):
    # 'weights' are the trainable parameters (like neural net weights)
    # We apply rotations to the qubit based on these weights.
    qml.RX(weights[0], wires=0)
    qml.RY(weights[1], wires=0)

    # THE OUTPUT layer: We measure the "spin" in the Z direction.
    # Result = +1 (State |0>) to -1 (State |1>)
    return qml.expval(qml.PauliZ(0))


In [ ]:
# 2. THE LOSS FUNCTION
# We want to force the qubit into the |1> state (value -1).
# So, our goal is to minimize the output value.
def loss_fn(params):
    prediction = quantum_model(params)
    return prediction # We want to make this number as low as possible (-1)


In [ ]:
# 3. THE OPTIMIZER (Gradient Descent)
# PennyLane automatically calculates the gradients of the quantum gates!
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# Initialize random "weights" (rotation angles)
# requires_grad=True tells PennyLane "Train this variable!"
weights = np.array([0.011, 0.012], requires_grad=True)

print(f"Initial Weights: {weights}")
print(f"Initial Output (should be near +1): {quantum_model(weights):.4f}")

print("\n--- TRAINING START ---")
for i in range(50):
    # The optimizer computes the gradient of the circuit and updates weights
    weights = opt.step(loss_fn, weights)

    if (i + 1) % 5 == 0:
        current_loss = loss_fn(weights)
        print(f"Step {i+1}: Loss = {current_loss:.4f}")

print("--- TRAINING END ---")
print(f"Optimized Weights: {weights}")
print(f"Final Output (should be near -1): {quantum_model(weights):.4f}")

In [ ]:
#We can draw the circuit to visualise it, but this isn't pennylanes strong suit
print(qml.draw(quantum_model)(weights))

# Proposed exercises

## Quantum Teleportation

In [ ]:
# 1. Setup the Universe (3 wires)
dev = qml.device("default.qubit", wires=["Message", "Alice", "Bob"])


In [ ]:
# --- 2. Helper Functions for the Presentation ---
def create_entangled_bridge():
    """Create a Bell pair between Alice and Bob."""
    # https://en.wikipedia.org/wiki/File:The_Hadamard-CNOT_transform_on_the_zero-state.png
    # TODO: Implement the circuit to create the entangled state

def alice_operations():
    """Alice interacts the message with her entangled qubit."""
    # CNOT from Message to Alice's entangled qubit
    # Todo:
    # Hadamard on the Message
    #Todo:

def bob_correction():
    """Bob uses Alice's 'classical' results to fix his state.
    In a real device, this happens after measurement.
    Here, we simulate the logic using controlled gates."""
    # If Alice's entangled qubit (Alice) is 1, apply X
    qml.CNOT(wires=["Alice", "Bob"])
    # If Alice's message qubit (Message) is 1, apply Z
    qml.CZ(wires=["Message", "Bob"])


In [ ]:
# --- 3. the main experiment ---
@qml.qnode(dev)
def teleportation_circuit(state_angles):
    # step a: prepare the state alice wants to send
    # we use arbitrary angles to create a unique unknown state |psi>
    qml.rx(state_angles[0], wires="message")
    qml.ry(state_angles[1], wires="message")

    # step b: create the shared resource (entanglement)
    create_entangled_bridge()

    # step c: alice performs her measurements/operations
    alice_operations()

    # step d: bob receives info and corrects his state
    bob_correction()

    # verification: return the state of bob's qubit
    return qml.state()


In [ ]:
# --- 4. running the experiment ---
# let's pick a random state for alice to send
angles = [0.5, 1.2]

# Run the teleportation
final_system_state = teleportation_circuit(angles)

# To verify, let's create what the state SHOULD look like
dev_verify = qml.device("default.qubit", wires=1)
@qml.qnode(dev_verify)
def target_state_circuit(angles):
    qml.RX(angles[0], wires=0)
    qml.RY(angles[1], wires=0)
    return qml.state()


In [ ]:
target_state = target_state_circuit(angles)

print("Teleportation Complete!")
# We look at the density matrix or state vector relative to Bob to verify overlap
# (Simplified verification for presentation purposes would be comparing measuring stats)
print(f"Target State Vector (first few terms): {target_state[:2]}")
# Note: Inspecting the full state vector of the 3-qubit system is complex,
# but Bob's subspace should match the target.

## Stock Price Predictor

**What we'll do:**
1. Load Tesla stock price data
2. Normalize the data for quantum processing
3. Train the model
4. Define the cost function
5. Train the model
6. Predict tomorrow's price
7. Visualize the results
---


#### <span style="color:red"> Disclaimer: This is a toy exercise and is not indicative of real stock prediction algorithms </span>

### 1. Loading the data

This is the script to get the data, but for the purposes of this notebook, it's not necessary to do the query

In [ ]:
# OPTION A: Real-world data (Requires 'pip install yfinance')
# !pip install yfinance
# import yfinance as yf
# ticker = yf.Ticker("TSLA")
# data = ticker.history(period="1mo")['Close'].values
# prices = np.array(data[-15:])


In [ ]:
# Tesla closing prices from December 2025
prices = np.array([
    430.14, 429.24, 446.74, 454.53, 455.00,
    439.58, 445.17, 451.45, 446.89, 458.96,
    475.31
])

In [ ]:
days = np.arange(len(prices))

In [ ]:
print(f"Loaded {len(prices)} days of stock data")
print(f"Price range: ${prices.min():.2f} - ${prices.max():.2f}")

---

## 2. Normalize the Data

We need to get our data to fit to these ranges:
- **Input (days)**: Normalized to [0, π]
- **Output (prices)**: Normalized to [-0.9, 0.9]


The formula we use in the cell below is called Min-Max Normalization:
<br> https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)


In [ ]:
# TODO: X_norm is the days normalized.
# Suggested formula is: MinMax(days) * np.pi
x_norm = None

In [ ]:
# Normalize Prices (Target) to range [-0.9, 0.9]
# Same formula as above, but with a different factor
p_min, p_max = prices.min(), prices.max()
#TODO:
y_norm = None

##### Hint for y_norm

###### To get the -0.9, you'll need to subtract -0.9 from the array values, so multiply with a y such that <br>
0 - 0.9 = -0.9 <br>
y - 0.9 = 0.9

##### Test Part 2

In [ ]:
print(f"Normalized input range: {x_norm.min():.4f} to {x_norm.max():.4f}")
print(f"Normalized output range: {y_norm.min():.4f} to {y_norm.max():.4f}")

In [ ]:
assert np.isclose(x_norm.min(), 0, atol=1e-10), f"x_norm minimum should be ~0, got {x_norm.min()}"
assert np.isclose(x_norm.max(), np.pi, atol=1e-10), f"x_norm maximum should be ~π, got {x_norm.max()}"

In [ ]:
assert np.isclose(y_norm.min(), -0.9, atol=1e-10), f"y_norm minimum should be ~-0.9, got {y_norm.min()}"
assert np.isclose(y_norm.max(), 0.9, atol=1e-10), f"y_norm maximum should be ~0.9, got {y_norm.max()}"

---

## 3. Build the Quantum Circuit

Our circuit has three parts:
1. **Data Encoding**: `RX` gate encodes the day as a rotation
2. **Trainable Model**: `Rot` gate with 3 learnable parameters
3. **Measurement**: Measures the qubit in the Z-basis (returns value between -1 and 1)


In [ ]:
dev = qml.device("default.qubit", wires=1)
n_params = 3
@qml.qnode(dev)
#TODO: change the _ to one of your inputs(either the day or the price)
# You should look at the next TODO, as that is where you will use the input
def circuit(params, _):
    # Data Encoding: Map day index to an angle
    # TODO: make an RX gate using the param you chose

    # Trainable Model: 3 rotation parameters
    # We'll use qml.Rot here: https://docs.pennylane.ai/en/stable/code/api/pennylane.Rot.html
    # TODO: implement qml.Rot using params
    # (assume params has the exact number of elems you need for qml.Rot)

    # Measurement: Returns value between -1 and 1
    return qml.expval(qml.PauliZ(0))

# Visualize the circuit
print("Circuit structure:")
print(qml.draw(circuit)(np.array([0.1, 0.2, 0.3]), 1.0))

In [ ]:
#Test your circuit
assert np.isclose(circuit(np.array([0.1, 0.2, 0.3]), 1.0), 0.5, atol=1e-1)

---

## 4. Define the Cost Function

We'll use Mean Squared Error (MSE) to measure how well our predictions match the actual prices. <br>
<br>If you never interacted with ML before, think of the cost function as <br>
**the way our model checks how good/bad the prediction of the model was**

In [ ]:
def cost(params, x, y):
    # 1. Get the list of prediction objects
    predictions_list = [circuit(params, xv) for xv in x]
    # 2. STACK them into a single Tensor (This keeps the gradient alive)
    predictions = qml.math.stack(predictions_list)
    # 3. Now compute MSE.
    return np.mean((predictions - y) ** 2)

---

## 5. Train the Model

We'll use the Adam optimizer to train our quantum circuit for 100 steps.

**A mathematical process that takes the cost and determines how to change the parameters to <span style="color:orange">try </span> to reduce the cost**

In [ ]:
np.random.seed(42)  # For reproducibility
params = np.random.uniform(0, np.pi, n_params, requires_grad =True)
# params = params.astype(np.float64)
# Initialize optimizer
opt = qml.AdamOptimizer(stepsize=0.1)

print(f"Training on {len(prices)} days of stock data...")
print("-" * 50)

# Training loop
for i in range(100):
    params = opt.step(lambda p: cost(p, x_norm, y_norm), params)
    if (i+1) % 20 == 0:
        current_cost = cost(params, x_norm, y_norm)
        print(f"Step {i+1:3d}: Cost = {current_cost:.6f}")

print("-" * 50)
print("Training complete!")

---

## 6. Predict Tomorrow's Price

Now let's use our trained model to predict the next day's closing price.

In [ ]:
# Predict for the next day (one step beyond our data)
# When we normalized our data, day 1 became 0 and the last day became pi.
next_day_norm = np.pi + (np.pi / (len(days) - 1))
# This is a close estimate, but the most correct way to do it is
# next_day_norm = x_norm[-1] + (x_norm[1] - x_norm[0])

In [ ]:
# Get normalized prediction
# TODO: call your circuit with the next_day_norm input
prediction_norm = circuit(params, next_day_norm)


In [ ]:
# Denormalize to get actual dollar price
# TODO: inverse of the steps we took to normalize the values
# Hint: we ended by subtracting 0.9 from the prediction
# Now we first need to add 0.9 to the prediction
predicted_price = ((prediction_norm + 0.9) / 1.8) * (p_max - p_min) + p_min

In [ ]:
print(f"📈 Predicted Stock Price for Day {len(days)+1}: ${predicted_price:.2f}")
print(f"   Last known price (Day {len(days)}): ${prices[-1]:.2f}")
print(f"   Change: ${predicted_price - prices[-1]:+.2f}")

---

## 7. Visualize Results

Let's plot the actual data, our quantum model's fit, and the prediction.

In [ ]:
# Calculate predictions for all training points
y_pred_norm = [circuit(params, x) for x in x_norm]

# Denormalize predictions back to dollar values
y_pred_real = [((y + 0.9) / 1.8) * (p_max - p_min) + p_min for y in y_pred_norm]

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(days, prices, color='black', s=100, label='Actual Data', zorder=3)
plt.plot(days, y_pred_real, color='blue', linewidth=2, label='Quantum Fit')
plt.scatter([len(days)], [predicted_price], color='red', marker='x',
            s=200, linewidth=3, label='Prediction', zorder=4)

plt.legend(fontsize=12)
plt.title("Quantum Stock Prediction (Simple Regression)", fontsize=14, fontweight='bold')
plt.xlabel("Day", fontsize=12)
plt.ylabel("Price ($)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## Summary

**What we accomplished:**
- Built a simple quantum regression model using PennyLane
- Trained it on daily values of Tesla stock prices
- Made a prediction for the next day
<br>
**Disclaimer:** <br>
- as you can see, the model predicts only a line (the same value no matter the input) <br>
this can be due to several reasons: <br>
- the model not having enough trainable params <br>
- not enough qubits <br>
- not enough data points <br>

### A working predictor


Now that we understand qml better, we can look at how to actually solve this problem

In [ ]:
import pennylane as qml
import numpy as np

# More qubits for richer representation
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def circuit(params, x):
    """
    Improved circuit with:
    - Multiple qubits
    - Feature map for better encoding
    - Multiple layers (depth)
    - Entanglement between qubits
    """
    n_qubits = 4
    n_layers = 3

    # === DATA ENCODING (Feature Map) ===
    # Encode input across multiple qubits with different transformations
    for i in range(n_qubits):
        # Notice each qubit is rotated by a different parameter
        qml.RX(x * (i + 1) * 0.5, wires=i)
        # We are rotating by an exponential function of the input, to make the effect more pronounced
        qml.RZ(x ** 2 * 0.1, wires=i)

    # === VARIATIONAL LAYERS ===
    param_idx = 0
    for layer in range(n_layers):
        # Rotation layer
        for i in range(n_qubits):
            # we rotate by a diff param for each qubit
            qml.Rot(params[param_idx],
                   params[param_idx + 1],
                   params[param_idx + 2],
                   wires=i)
            param_idx += 3

        # Entangling layer (creates correlations between qubits)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])  # Circular entanglement

    # === MEASUREMENT ===
    # Measure multiple qubits and combine
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))


In [ ]:
# Calculate number of parameters needed
n_qubits = 4
n_layers = 3
n_params = n_qubits * 3 * n_layers  # 36 parameters

In [ ]:
# Test
params = np.random.randn(n_params) * 0.1
print("Circuit structure:")

In [ ]:
# Repeat code, cause functions are overrated
# Calculate predictions for all training points
y_pred_norm = [circuit(params, x) for x in x_norm]

# Denormalize predictions back to dollar values
y_pred_real = [((y + 0.9) / 1.8) * (p_max - p_min) + p_min for y in y_pred_norm]

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(days, prices, color='black', s=100, label='Actual Data', zorder=3)
plt.plot(days, y_pred_real, color='blue', linewidth=2, label='Quantum Fit')
plt.scatter([len(days)], [predicted_price], color='red', marker='x',
            s=200, linewidth=3, label='Prediction', zorder=4)

plt.legend(fontsize=12)
plt.title("Quantum Stock Prediction (Simple Regression)", fontsize=14, fontweight='bold')
plt.xlabel("Day", fontsize=12)
plt.ylabel("Price ($)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()